<a href="https://colab.research.google.com/github/Nischal2015/ncit-workshop/blob/main/1-basic_agents/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 1
### Writing your very first agent
### With a "College Assignment" style example

### Only run this if you are using Google Colab

In [ ]:
!pip install langsmith["openai-agents"] openai-agents langchain langchain-openai

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "ncit-workshop"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

### Imports

In [ ]:
from agents import Agent, Runner, set_trace_processors
from langsmith.wrappers import OpenAIAgentsTracingProcessor
import nest_asyncio

nest_asyncio.apply()
set_trace_processors([OpenAIAgentsTracingProcessor()])

### Make a simple agent with name, instructions and model

In [ ]:
agent = Agent(
    name="Simple Agent",
    instructions="You are a helpful assistant for doing my college assignments.",
    model="gpt-4.1-mini",
)

#### Prompt to perform a certain task (generate a simple C code to print asterisk pattern)

In [ ]:
input = """
Please come up with C code to print the asterisk(*) pattern for n = <some number>.:

This is the pattern for n = 4:

    *
   ***
  *****
 *******

Give the output in markdown format with proper code block.

"""

In [ ]:
async def run_agent(agent: Agent, input: str) -> str:
    result = await Runner.run(starting_agent=agent, input=input)
    return result.final_output

In [ ]:
import asyncio
from IPython.display import Markdown

output = asyncio.run(run_agent(agent, input))
Markdown(output)

In [ ]:
result = asyncio.run(run_agent(agent, input))
Markdown(result)

### Structured Output

In [ ]:
from pydantic import BaseModel
from typing import Literal


# Structure for the agent's output
class GoOutDecision(BaseModel):
    """Structure for the agent's output regarding going out decision."""

    decision: Literal["go_out", "stay_in", "compromise"]
    """Decides whether to go out, stay in, or find a compromise."""

    confidence_level: Literal["high", "low", "medium"]
    """Confidence level of the decision."""

    main_reason: str
    """Main reason for the decision."""

    risk: list[str]
    """List of risks associated with the decision."""

    suggested_compromise: str
    """Suggested compromise if applicable."""

In [ ]:
# Go out decision agent
go_out_agent = Agent(
    model="gpt-4.1-mini",
    name="Friday Night Decision Agent",
    instructions=(
        "You help students decide whether to go out or stay in on Friday nights. "
        "Be realistic, friendly and little playful, but not reckless. "
        "Take into account energy level, responsibilities, and timing."
    ),
    output_type=GoOutDecision,
)

In [ ]:
# Input prompt
student_input = (
    "My friends want me to go out tonight, but I have an exam at 9am tomorrow "
    "and I'm already pretty tired. What should I do?"
)

In [ ]:
set_trace_processors([OpenAIAgentsTracingProcessor(name="Go Out Workflow")])

result = await Runner.run(starting_agent=go_out_agent, input=student_input)
final_output = result.final_output
final_output

In [ ]:
import json

# print(json.dumps(final_output.model_dump(), indent=2))
final_output.model_dump()

### Langchain Agents

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,
    max_completion_tokens=1000,
    timeout=30,
)

agent = create_agent(
    model=model,
    name="Friday Night Decision Agent",
    system_prompt=(
        "You help students decide whether to go out or stay in on Friday nights. "
        "Be realistic, friendly and little playful, but not reckless. "
        "Take into account energy level, responsibilities, and timing."
    ),
    response_format=GoOutDecision,
)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": student_input}]})
result

In [ ]:
json.loads(result["messages"][-1].content)